<a href="https://colab.research.google.com/github/harjeet88/LLM_experiemnts/blob/main/LLMs_course/week1/day2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [2]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [3]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]


In [4]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [9]:
!curl -LO https://ollama.ai/install.sh
!bash install.sh



  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13281    0 13281    0     0  62299      0 --:--:-- --:--:-- --:--:-- 62352
>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [28]:
!nohup ollama serve > ollama.log 2>&1 &

In [17]:
# Let's just make sure the model is loaded

!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling dde5aa3fc5ff...   0% ▕▏    0 B/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   0% ▕▏    0 B/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   3% ▕▏  64 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   4% ▕▏  74 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   5% ▕▏ 104 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   8% ▕▏ 155 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   8% ▕▏ 159 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...  10% ▕▏ 194 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...  12% ▕▏ 250 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...  13% ▕▏ 254 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...  14% ▕▏ 285 MB/2.0 GB  284 MB/s      6spulling manife

In [20]:
!pip install ollama

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00


In [23]:
# If this doesn't work for any reason, try the 2 versions in the following cells
# And double check the instructions in the 'Recap on installation of Ollama' at the top of this lab
# And if none of that works - contact me!

response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

KeyboardInterrupt: 

In [29]:
!curl http://127.0.0.1:11434/

Ollama is running

In [34]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.5/474.5 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.6/345.6 kB 25.5 MB/s eta 0:00:00


In [30]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

Generative AI has numerous business applications across various industries, including:

1. **Content Generation**: AI-powered tools can generate high-quality content such as articles, social media posts, product descriptions, and more. This saves time and resources for content creation teams.
2. **Product Design**: Generative AI can be used to design new products, such as 3D models, logos, and packaging designs. This enables companies to quickly explore different design options without the need for extensive prototyping or human input.
3. **Marketing Automation**: AI-generated content can be used to personalize marketing campaigns, create targeted ads, and optimize email marketing campaigns.
4. **Customer Service Chatbots**: Generative AI-powered chatbots can provide 24/7 customer support, answer frequently asked questions, and route complex issues to human representatives.
5. **Image and Video Editing**: AI algorithms can generate new images or edit existing ones, enabling companies t

In [31]:
!ollama pull deepseek-r1:1.5b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling aabd4debf0c8...   0% ▕▏    0 B/1.1 GB                  pulling manifest 
pulling aabd4debf0c8...   0% ▕▏    0 B/1.1 GB                  pulling manifest 
pulling aabd4debf0c8...   0% ▕▏ 2.3 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8...   3% ▕▏  37 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8...  11% ▕▏ 123 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8...  12% ▕▏ 136 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8...  17% ▕▏ 189 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8...  23% ▕▏ 260 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8...  26% ▕▏ 291 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8...  30% ▕▏ 332 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8...  36% ▕▏ 406 MB/1.1 GB                  pulling manife

In [35]:
# There's actually an alternative approach that some people might prefer
# You can use the OpenAI client python library to call Ollama:

from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages
)

print(response.choices[0].message.content)

APITimeoutError: Request timed out.

In [32]:
# This may take a few minutes to run! You should then see a fascinating "thinking" trace inside <think> tags, followed by some decent definitions

response = ollama_via_openai.chat.completions.create(
    model="deepseek-r1:1.5b",
    messages=[{"role": "user", "content": "Please give definitions of some core concepts behind LLMs: a neural network, attention and the transformer"}]
)

print(response.choices[0].message.content)


NameError: name 'ollama_via_openai' is not defined